# Transformer with non-trivial phase shift and tap ratio

This example is a copy of pandapower's minimal example.

In [ ]:
import numpy as np
import pandas as pd

import pypsa

In [ ]:
network = pypsa.Network()

network.add("Bus", "MV bus", v_nom=20, v_mag_pu_set=1.02)
network.add("Bus", "LV1 bus", v_nom=0.4)
network.add("Bus", "LV2 bus", v_nom=0.4)

network.add(
    "Transformer",
    "MV-LV trafo",
    type="0.4 MVA 20/0.4 kV",
    bus0="MV bus",
    bus1="LV1 bus",
)
network.add(
    "Line", "LV cable", type="NAYY 4x50 SE", bus0="LV1 bus", bus1="LV2 bus", length=0.1
)
network.add(
    "Generator", "External Grid", bus="MV bus", control="Slack", marginal_cost=10
)
network.add("Load", "LV load", bus="LV2 bus", p_set=0.1, q_set=0.05)

In [ ]:
def run_pf():
    network.lpf()
    network.pf(use_seed=True)
    return pd.DataFrame(
        {
            "Voltage Angles": network.buses_t.v_ang.loc["now"] * 180.0 / np.pi,
            "Volate Magnitude": network.buses_t.v_mag_pu.loc["now"],
        }
    )

In [ ]:
run_pf()

In [ ]:
network.transformers.tap_position = 2
run_pf()

In [ ]:
network.transformers.tap_position = -2
run_pf()

Now play with tap changer on LV side

In [ ]:
new_trafo_lv_tap = network.transformer_types.loc[["0.4 MVA 20/0.4 kV"]]
new_trafo_lv_tap.index = ["New trafo"]
new_trafo_lv_tap.tap_side = 1
new_trafo_lv_tap.T

In [ ]:
network.transformer_types = pd.concat([network.transformer_types, new_trafo_lv_tap])
network.transformers.type = "New trafo"
network.transformers.tap_position = 2
run_pf()

In [ ]:
network.transformers.T

In [ ]:
network.transformers.tap_position = -2
run_pf()

Now make sure that the phase shift is also there in the LOPF

In [ ]:
network.generators.p_nom = 1.0
network.lines.s_nom = 1.0
network.optimize()
pd.DataFrame(
    {
        "Voltage Angles": network.buses_t.v_ang.loc["now"] * 180.0 / np.pi,
        "Volate Magnitude": network.buses_t.v_mag_pu.loc["now"],
    }
)